In [1]:
!pip install peft transformers datasets bitsandbytes torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 11.7 MB/s eta 0:00:00


In [2]:
pip install pypdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from huggingface_hub import login

login(token="YOUR_HUGGINGFACE_TOKEN HERE # Replace with your actual token")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import bitsandbytes
from peft import get_peft_model, LoraConfig, TaskType
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

config = PeftConfig.from_pretrained("abhi317/final_llama")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct",device_map="auto",load_in_4bit=True)
model = PeftModel.from_pretrained(base_model, "abhi317/final_llama")


2024-06-24 08:33:05.510736: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 08:33:05.510839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 08:33:05.642026: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


adapter_config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

In [6]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

pdf_path = "/kaggle/input/mltopbooks/dokumen.pub_natural-language-processing-with-transformers-revised-edition-1098136799-9781098136796-9781098103248.pdf"
text_data = extract_text_from_pdf(pdf_path)

In [7]:
import re

def preprocess_text(text):
    # Example preprocessing steps
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

In [8]:
def prepare_training_data(text):
    # Split the text into chunks (e.g., paragraphs or specific lengths)
    # This is task-specific; adjust as needed
    chunks = text.split('\n\n')  # Assuming paragraphs are separated by double newlines
    input_texts = ["summarize: " + chunk for chunk in chunks]
    target_texts = input_texts # Replace with actual summaries if available
    return input_texts, target_texts

preprocessed_text = preprocess_text(text_data)

input_texts, target_texts = prepare_training_data(preprocessed_text)

In [9]:
#   input_texts[0]

In [10]:




from datasets import Dataset

tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    inputs = tokenizer(examples["input_texts"], padding="max_length", truncation=True)
    targets = tokenizer(examples["target_texts"], padding="max_length", truncation=True)
    inputs["labels"] = targets["input_ids"]
    return inputs


config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,   # Task type for causal language modeling
    r=8,                            # Low-rank dimension
    lora_alpha=64,                  # Scaling factor
    lora_dropout=0.5,               # Dropout rate
    target_modules=["q_proj", "v_proj","k_proj","dense"],# Target specific modules for adaptation
)


model = get_peft_model(model, config)


# Create dataset
dataset = Dataset.from_dict({"input_texts": input_texts, "target_texts": target_texts})
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./final_llama",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    num_train_epochs=50,
    save_steps=500,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    push_to_hub=True,
    logging_first_step=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

# Train the model
trainer.train()

trainer.push_to_hub("DONE")

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


UsageError: api_key not configured (no-tty). call wandb.login(key=[your_api_key])